# Лабораторная работа No7
## Задача:
Самостоятельно сгенерируйте 1 000 авторов и книги по каждому автору в диапазоне от 2 до 5, используя Python библиотеку Faker.

### Описание таблиц
Имеется часть модель данных для управления библиотекой (из Лабораторной работы No6).
### Таблица “authors” (Авторы)
Хранит информацию об авторах книг.

| Столбец    | Описание                                        |
|------------|-------------------------------------------------|
| author_id  | Первичный ключ, уникальный идентификатор автора |
| first_name | Имя автора (обязательное поле)                  |
| last_name  | Фамилия автора (обязательное поле)              |
| birth_date | Дата рождения автора (может быть NULL)          |
| country    | Страна происхождения автора (может быть NULL)   |

### Таблица “books” (Книги)
Содержит данные о книгах в библиотеке.

| Столбец          | Описание                                                     |
|------------------|--------------------------------------------------------------|
| book_id          | Первичный ключ, уникальный идентификатор книги               |
| title            | Название книги (обязательное поле)                           |
| author_id        | Внешний ключ, ссылается на `authors.author_id`               |
| publication_year | Год публикации книги                                         |
| isbn             | Уникальный ISBN книги                                        |
| genre            | Жанр книги (например, "Фэнтези", "Научная литература")       |
| page_count       | Количество страниц в книге                                   |
| available_copies | Количество доступных для выдачи экземпляров (по умолчанию 1) |

### SQL
```sql
CREATE TABLE authors (
author_id SERIAL PRIMARY KEY,
first_name VARCHAR(100) NOT NULL,
last_name VARCHAR(100) NOT NULL,
birth_date DATE,
country VARCHAR(100)
);

CREATE TABLE books (
book_id SERIAL PRIMARY KEY,
title VARCHAR(255) NOT NULL,
author_id INTEGER REFERENCES authors(author_id),
publication_year INTEGER,
isbn VARCHAR(20) UNIQUE,
genre VARCHAR(100),
page_count INTEGER,
available_copies INTEGER DEFAULT 1
);
```

In [171]:
import psycopg2
import random
from faker import Faker
from tabulate import tabulate

fake = Faker('ru_RU')
data_count = 1000

### Подключение к PostgreSQL
connection = psycopg2.connect(
    host="localhost",
    port="5432",
    database="",
    user="",
    password="",
)
### Создание курсора
cursor = connection.cursor()

print("Подключение к PostgreSQL успешно.")

Подключение к PostgreSQL успешно.


## Создание схемы

In [172]:
cursor.execute("CREATE SCHEMA IF NOT EXISTS ivan_patakin;")
connection.commit()
cursor.execute("SET search_path TO ivan_patakin;")

print("Схема 'ivan_patakin' создана. Используется в текущем сеансе.")

Схема 'ivan_patakin' создана. Используется в текущем сеансе.


## Создаем таблицы

In [173]:
tables = [
    """
    CREATE TABLE authors (
        author_id SERIAL PRIMARY KEY,
        first_name VARCHAR(100) NOT NULL,
        last_name VARCHAR(100) NOT NULL,
        birth_date DATE,
        country VARCHAR(100)
    );
    """,
    """
    CREATE TABLE books (
        book_id SERIAL PRIMARY KEY,
        title VARCHAR(255) NOT NULL,
        author_id INTEGER REFERENCES authors(author_id),
        publication_year INTEGER,
        isbn VARCHAR(20) UNIQUE,
        genre VARCHAR(100),
        page_count INTEGER,
        available_copies INTEGER DEFAULT 1
    );
    """
]

for table in tables:
    cursor.execute(table)
connection.commit()

print("Таблицы успешно созданы.")

Таблицы успешно созданы.


## Генерация данных для таблиц
### Генерация авторов

In [174]:
authors_data = []
countries = ["Россия", "США", "Великобритания", "Франция", "Германия", "Япония", "Китай", "Индия", "Канада", "Австралия"]

for _ in range(data_count):
    first_name = fake.first_name()
    last_name = fake.last_name()
    birth_date = fake.date_of_birth(minimum_age=20, maximum_age=100)
    country = random.choice(countries)
    authors_data.append((first_name, last_name, birth_date, country))

cursor.executemany(
    "INSERT INTO authors (first_name, last_name, birth_date, country) VALUES (%s, %s, %s, %s)",
    authors_data
)
connection.commit()
print("Авторы: данные сгенерированы и загружены.")

Авторы: данные сгенерированы и загружены.


### Генерация книг

In [175]:
books_data = []
genres = ["Фэнтези", "Детектив", "Роман", "Научная литература", "История", "Приключения", "Фантастика", "Поэзия", "Детская литература", "Биография"]

for author_id in range(1, data_count + 1):
    num_books = random.randint(2, 5)
    for _ in range(num_books):
        title = fake.catch_phrase()
        publication_year = random.randint(1950, 2023)
        isbn = f"978-{random.randint(0, 9)}-{random.randint(10000, 99999)}-{random.randint(100, 999)}-{random.randint(0, 9)}"
        genre = random.choice(genres)
        page_count = random.randint(50, 1500)
        available_copies = random.randint(1, 10)
        books_data.append((title, author_id, publication_year, isbn, genre, page_count, available_copies))

cursor.executemany(
    "INSERT INTO books (title, author_id, publication_year, isbn, genre, page_count, available_copies) VALUES (%s, %s, %s, %s, %s, %s, %s)",
    books_data
)
connection.commit()
print("Книги: данные сгенерированы и загружены.")

Книги: данные сгенерированы и загружены.


### Вывод итогового количества записей в таблицах

In [176]:
cursor.execute("SELECT COUNT(*) FROM authors")
authors_count = cursor.fetchone()[0]

cursor.execute("SELECT COUNT(*) FROM books")
books_count = cursor.fetchone()[0]

print("\n=== Итоговое количество записей в таблицах ===")
print(f"Авторы: {authors_count}")
print(f"Книги: {books_count}")


=== Итоговое количество записей в таблицах ===
Авторы: 1000
Книги: 3511


### Вывод данных из таблиц
Для упрощения вывод ограничен 10 записями из таблиц
#### Вывод данных из таблицы authors

In [177]:
cursor.execute("SELECT * FROM authors LIMIT 10;")
authors = cursor.fetchall()
headers = [desc[0] for desc in cursor.description]

print(tabulate(authors, headers=headers, tablefmt="grid"))

+-------------+--------------+-------------+--------------+-----------+
|   author_id | first_name   | last_name   | birth_date   | country   |
+=============+==============+=============+==============+===========+
|           1 | Исай         | Бирюков     | 1936-03-07   | Канада    |
+-------------+--------------+-------------+--------------+-----------+
|           2 | Селиверст    | Маслова     | 1997-04-18   | Россия    |
+-------------+--------------+-------------+--------------+-----------+
|           3 | Парфен       | Веселов     | 1975-10-31   | Индия     |
+-------------+--------------+-------------+--------------+-----------+
|           4 | Вячеслав     | Дмитриева   | 1985-10-15   | Индия     |
+-------------+--------------+-------------+--------------+-----------+
|           5 | Сила         | Блохин      | 1982-03-01   | Россия    |
+-------------+--------------+-------------+--------------+-----------+
|           6 | Радим        | Суворов     | 1928-07-22   | Герм

#### Вывод данных из таблицы books

In [178]:
cursor.execute("SELECT * FROM books LIMIT 10;")
books = cursor.fetchall()
headers = [desc[0] for desc in cursor.description]

print(tabulate(books, headers=headers, tablefmt="grid"))

+-----------+--------------------------------------------------------------+-------------+--------------------+-------------------+--------------------+--------------+--------------------+
|   book_id | title                                                        |   author_id |   publication_year | isbn              | genre              |   page_count |   available_copies |
+===========+==============================================================+=============+====================+===================+====================+==============+====================+
|         1 | Межгрупповое и заметное оборудование                         |           1 |               1955 | 978-1-34487-261-7 | Поэзия             |         1059 |                  4 |
+-----------+--------------------------------------------------------------+-------------+--------------------+-------------------+--------------------+--------------+--------------------+
|         2 | Управляемая и приоритетная нейронная сеть

#### Вывод данных колличества книг у автора

In [179]:
cursor.execute("""
CREATE VIEW author_book_count AS
SELECT
    a.author_id,
    a.first_name,
    a.last_name,
    COUNT(b.book_id) AS book_count
FROM
    authors a
LEFT JOIN
    books b
ON
    a.author_id = b.author_id
GROUP BY
    a.author_id, a.first_name, a.last_name;
""")

connection.commit()

cursor.execute("SELECT * FROM author_book_count LIMIT 10;")
books = cursor.fetchall()
headers = [desc[0] for desc in cursor.description]

print(tabulate(books, headers=headers, tablefmt="grid"))


+-------------+--------------+-------------+--------------+
|   author_id | first_name   | last_name   |   book_count |
+=============+==============+=============+==============+
|         652 | Будимир      | Белозеров   |            5 |
+-------------+--------------+-------------+--------------+
|         273 | Твердислав   | Трофимов    |            4 |
+-------------+--------------+-------------+--------------+
|          51 | Ратибор      | Захаров     |            5 |
+-------------+--------------+-------------+--------------+
|         951 | Святополк    | Селезнев    |            4 |
+-------------+--------------+-------------+--------------+
|         839 | Эмиль        | Вишняков    |            5 |
+-------------+--------------+-------------+--------------+
|          70 | Мир          | Горбачева   |            4 |
+-------------+--------------+-------------+--------------+
|         350 | Радислав     | Степанов    |            3 |
+-------------+--------------+----------

## Удаление таблиц

In [180]:
tables_to_drop = [
    'authors',
    'books'
]

cursor.execute("DROP VIEW IF EXISTS author_book_count;")

for table in tables_to_drop:
    cursor.execute(f"DROP TABLE IF EXISTS {table} CASCADE;")

connection.commit()
print("Таблицы успешно удалены.")

Таблицы успешно удалены.


### Закрытие соединения

In [181]:
cursor.close()
connection.close()

print("Соединение с PostgreSQL закрыто.")

Соединение с PostgreSQL закрыто.
